## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Ponta Do Sol,PT,72.23,clear sky,32.6667,-17.1000,Hotel do Campo
1,1,La Ronge,CA,82.44,overcast clouds,55.1001,-105.2842,Northland Motor Hotel
2,2,Yinchuan,CN,73.71,light rain,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
3,3,Kavieng,PG,83.48,light rain,-2.5744,150.7967,Nusa Island Retreat
4,4,Mecca,SA,85.64,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"


#### Input statements are written to prompt the customer for their minimum and maximum temperature preferences

In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


#### A new DataFrame is created based on the minimum and maximum temperature, and empty rows are dropped. 

In [6]:
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Ponta Do Sol,PT,72.23,clear sky,32.6667,-17.1000,Hotel do Campo
1,1,La Ronge,CA,82.44,overcast clouds,55.1001,-105.2842,Northland Motor Hotel
2,2,Yinchuan,CN,73.71,light rain,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
3,3,Kavieng,PG,83.48,light rain,-2.5744,150.7967,Nusa Island Retreat
4,4,Mecca,SA,85.64,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
5,5,Wanning,CN,80.78,overcast clouds,18.8003,110.3967,Paradise Hotel
6,6,Prainha,BR,77.90,broken clouds,-1.8000,-53.4800,Casa Rocha
7,7,Kawalu,ID,71.49,overcast clouds,-7.3817,108.2082,KOSAN UNGU UNSIK
8,8,Changji,CN,74.19,clear sky,44.0167,87.3167,Lvye Holiday Resort
9,9,Atuona,PF,77.67,few clouds,-9.8000,-139.0333,Villa Enata


In [7]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                267
City                   267
Country                267
Max Temp               267
Current Description    267
Lat                    267
Lng                    267
Hotel Name             267
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(inplace = True) 

#### The hotel name is retrieved and added to the DataFrame, and the rows that don’t have a hotel name are dropped.

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,72.23,clear sky,32.6667,-17.1000,
1,La Ronge,CA,82.44,overcast clouds,55.1001,-105.2842,
2,Yinchuan,CN,73.71,light rain,38.4681,106.2731,
3,Kavieng,PG,83.48,light rain,-2.5744,150.7967,
4,Mecca,SA,85.64,broken clouds,21.4267,39.8261,
5,Wanning,CN,80.78,overcast clouds,18.8003,110.3967,
6,Prainha,BR,77.90,broken clouds,-1.8000,-53.4800,
7,Kawalu,ID,71.49,overcast clouds,-7.3817,108.2082,
8,Changji,CN,74.19,clear sky,44.0167,87.3167,
9,Atuona,PF,77.67,few clouds,-9.8000,-139.0333,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,72.23,clear sky,32.6667,-17.1000,Hotel do Campo
1,La Ronge,CA,82.44,overcast clouds,55.1001,-105.2842,Northland Motor Hotel
2,Yinchuan,CN,73.71,light rain,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
3,Kavieng,PG,83.48,light rain,-2.5744,150.7967,Nusa Island Retreat
4,Mecca,SA,85.64,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
...,...,...,...,...,...,...,...
262,Karnal,IN,79.54,broken clouds,29.6833,76.9833,Hotel Deventure
263,West Bay,KY,89.08,few clouds,19.3667,-81.4167,Calypso Cove
264,Arani,IN,81.45,overcast clouds,12.6667,79.2833,KPK LODGE
265,Tiarei,PF,82.42,overcast clouds,-17.5333,-149.3333,Le Rocher de Tahiti


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True) 
len(hotel_df)

267

#### The DataFrame is exported as a CSV file into the Vacation_Search folder

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

#### Create pop up info

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

#### A marker layer map with pop-up markers for the cities in the vacation DataFrame is created.

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))